In [ ]:
from importlib import reload
import ipynb.fs.full.notebook_viewer
reload(ipynb.fs.full.notebook_viewer)
from ipynb.fs.full.notebook_viewer import Viewer, Manager, root, fname2id, id2fname, split_dump, Filter, prepare, tqdm_load
import pickle
from dotdict import dotdict
from functools import reduce

from typing import *
from random import Random
from pathlib import Path
import json
from collections import defaultdict
from vivid.utils import timer
from tqdm import tqdm
from functools import lru_cache
import numpy as np
from PIL import Image
import imagesize

In [ ]:
readable_ori =  set((root/"danbooru2020/dump_readable_ori/0").read_text().strip().split())
missing2x = {"4005075","685111","3554125","3004206","1783239","528288","1717363","4268380","750472","1106509","2072514","2072517","3185545","4039613","3051636","4089762","800782","2515799","1195810","4260855"}
missingsolov2 = {'1084454','2334893','2829693','2832773','2945896','3608576','3657998','3701002','3822349','4054580'}

fire_eggs = dict()
for k in "duplicates is_deleted no_humans not-image photo text_only_page".split():
    fire_eggs[k] = set((root/f"danbooru2020/Danbooru2019/reduce/{k}.txt").read_text().strip().split())
    
ge512 =  set((root/"danbooru2020/ge512.ids").read_text().strip().split())
ge1024 =  set((root/"danbooru2020/ge1024.ids").read_text().strip().split())

i = 2
if not "ids" in globals():
    with timer(prefix="read_id_pkl"): # 311[s]
        ids = tqdm_load(root/f"danbooru2020/{i}_id.pkl")
if not "tags" in globals():
    with timer(prefix="read_tag_pkl"): # 58[s]
        tags = tqdm_load(root/f"danbooru2020/{i}_tag.pkl")

In [ ]:
@lru_cache(maxsize=None)
def t(s):
    if "|" in s:
        retval = reduce(set.__or__, map(tags.get, s.split("|")))
    else:
        retval = reduce(set.__and__, map(tags.get, s.split("&")))
    return retval

@lru_cache(maxsize=None)
def load_json(fname):
    fname = str(fname)
    retval = dotdict({k.replace("detection_", "").replace("bbox_", "").replace("class_", ""): np.array(v) for k, v in json.loads(Path(fname).read_text()).items()})
    retval["fname"] = fname
    retval["id"] = fname2id(fname)
    return retval

@lru_cache(maxsize=None)
def load_pkl(fname):
    fname = str(fname)
    with open(fname, "rb") as f:
        retval = dotdict(pickle.load(f))
    return retval

@lru_cache(maxsize=None)
def load_arr(fname):
    img = Image.open(fname)
    arr = np.array(img)
    return arr

In [ ]:
def id2original(_id):
    return id2fname(_id, prefix="original", ext=lambda _id: ids[_id]["file_ext"])

def id2original2x(_id):
    if _id in tags["highres"]:
        return id2original(_id)
    return id2fname(_id, prefix="original2x", ext=lambda _: "png")

def id2original2xwhite(_id):
    if _id in tags["highres"]:
        return id2fname(_id, prefix="originalwhite", ext=lambda _: "png")
    return id2fname(_id, prefix="original2xwhite", ext=lambda _: "png")

def id2whitewaist(_id):
    return id2fname(_id, prefix="whitewaist")

def id2json(_id):
    return id2fname(_id, prefix="solo_json", bucket=lambda _: ".", ext=lambda _: "json")

def id2pkl(_id):
    return id2fname(_id, prefix="yaas/solov2out", ext=lambda _: "pkl")

def id2score(_id):
    return int(ids[_id]["score"])

def grouping(fnames, load=load_json, f=lambda x: np.where(x.label == 1)[0]):
    r = dotdict()
    r.fnames = tuple(map(str, fnames))
    r.zero = set()
    r.one = set()
    r.multi = set()
    pbar =  tqdm(r.fnames)
    for i, fname in enumerate(pbar):
        _id = fname2id(fname)
        x = load(fname)
        r[_id] = x
        fx = f(x)
        if len(fx) == 0:
            r.zero.add(_id)
        elif len(fx) == 1:
            r.one.add(_id)
        else:
            r.multi.add(_id)
        if i+1 == len(r.fnames):
            pbar.set_description(str(len(r.one)/(i+1)*100)[:2]+"%")
    return r

def scoring(keys, f=id2score):
    cnt = defaultdict(set)
    for _id in keys:
        cnt[f(_id)].add(_id)
    cnt = list(map(list, cnt.items()))
    cnt.sort()
    for i in reversed(range(len(cnt)-1)):
        cnt[i][1] |= cnt[i+1][1]
    for i, (score, arr) in enumerate(cnt):
        print(score, len(arr))
        if i == 10: break
    cnt = dict(cnt)
    return cnt

def created_at(keys):
    return scoring(keys, f=lambda _id: int(ids[_id]['created_at'].split("-")[0]))

In [ ]:
solo = t("1girl|solo|1boy")-t("1girl&1boy")-t("2boys|2girls|3boys|3girls|4boys|4girls|5boys|5girls|6+boys|6+girls|multiple_boys|multiple_girls|multiple_views|variations")
exc0 = t('|'.join([
    "no_humans|arachne|pokemon_(creature)|octoling|monster_girl|isabelle_(animal_crossing)|animal_nose|animal_hug",
    "robot|cyborg|mecha_musume|machinery|doll_joints|crewmate_(among_us)|kamen_rider_faiz|olimar",
    "red_skin|blue_skin|purple_skin|wrinkled_skin|green_skin|two-tone_skin",
    "monochrome|lineart|text_only_page|photo-referenced|3d|unconventional_media|comic",
    "photo_(medium)|reference_photo_inset|reference_photo|photorealistic",
    "full_armor|weapon|chibi|cannon|mask|eyepatch",
]))

In [ ]:
fl = Filter()
fl["solo"]         = solo&readable_ori - missing2x - missingsolov2
fl["exc0"]         = fl[-1]-exc0
fl["standing"]     = fl[-1]&tags["standing"]
fl["ge512"]        = fl[-1]&ge512
face     = grouping(map(id2json, fl["ge512"]), load=load_json, f=lambda x: np.where(x.label == 1)[0])
instance = grouping(map(id2pkl,  fl["ge512"]), load=load_pkl,  f=lambda x: x.pred_classes)
fl["faceinstance"] = face.one&instance.one
fl["censored"]     = fl[-1]-tags["censored"]
fl["safe"]         = {_id for _id in fl["censored"] if ids[_id]["rating"] == "s"}
fl["questionable"] = {_id for _id in fl["censored"] if ids[_id]["rating"] == "q"}
fl["explicit"]     = {_id for _id in fl["censored"] if ids[_id]["rating"] == "e"}
fl["full_body"]    = fl["safe"]&tags["full_body"]

In [ ]:
exc1 = t('|'.join([
    "zoom_layer",
    "back|ass|from_behind|from_above|bent_over|standing_split|dutch_angle",
    "car|bicycle|translation_request|mystery_skulls",
    "fox_tail|otter_costume|sonic_the_hedgehog|sonic_the_hedgehog",
    "floating_head|cyclops|fat|electric_guitar",
]))|reduce(set.__or__, fire_eggs.values())
fl["out"]          = fl["safe"]-exc1

In [ ]:
def th_option(fname, dname):
    return f"/home/natsuki/waifu2x/waifu2x.lua -m scale -i {fname} -o {dname}".split()
options = list()
for _id in tqdm(list((fl["ge512"]-tags["highres"])|face.zero)):
    option = " ".join(th_option(_id, id2fname(id2original(_id), prefix="original2x")))
    options.append(option)
split_dump(options, "option_2x")

In [ ]:
options_convert = list()
options_mogrify = list()
for i, _id in enumerate(tqdm(instance.one)):
    srcname = id2original2xwhite(_id)
    maskname  = id2fname(_id, prefix="yaas/solov2out")
    dstname = id2fname(_id, prefix="original2xwhitesolov2")
    options_convert.append( f"-quality 100 {srcname} {maskname} -compose CopyOpacity -composite {dstname}" )
    options_mogrify.append( f"-quality 100 -background white -flatten PNG24:{dstname}")
    mask = instance[_id].pred_masks[0]
    H, W = mask.shape
    arr = 255*np.ones((H, W, 4), dtype="uint8")
    arr[:,:,3] = 255*mask
    img = Image.fromarray(arr)
    img = Image.fromarray((255*mask).astype("uint8")).convert("1")
    img.save(maskname)
split_dump(options_convert, "option_original2xwhitesolov2_convert")
split_dump(options_mogrify, "option_original2xwhitesolov2_mogrify")

In [ ]:
def convert_option(fname, dname, xc=0.5, yc=0.28, yw=0.235, size=512):
    _id = fname2id(fname)
    j = load_json(id2json(_id))
    src = dotdict()
    dst = dotdict()
    src.X, src.Y = imagesize.get(fname)
    i = np.where(j.label==1)[0][0]
    src.Xmin = src.X*j.xmin[i]
    src.Xmax = src.X*j.xmax[i]
    src.Ymin = src.Y*j.ymin[i]
    src.Ymax = src.Y*j.ymax[i]
    src.Ywid = src.Ymax-src.Ymin
    src.Xcen = (src.Xmax+src.Xmin)/2
    src.Ycen = (src.Ymax+src.Ymin)/2
    dst = dotdict()
    dst.Xcen = size*xc
    dst.Ycen = size*yc
    dst.Ywid = size*yw
    z = dst.Ywid/src.Ywid
    option = [
        "-background", "white",
        "-resize", f"{round(100*z)}%",
        "-crop", f"{size}x{size}+{round(z*src.Xcen-dst.Xcen)}+{round(z*src.Ycen-dst.Ycen)}!",
        "-flatten", "-strip",
        "-quality", "100",
        f"{fname}", f"PNG24:{dname}",
    ]
    return option
options = list()
for _id in tqdm(fl["faceinstance"]):
    option = " ".join(convert_option(id2fname(_id, prefix="original2xwhitesolov2"), id2fname(_id, prefix="whitewaist")))
    options.append(option)
split_dump(options, "option_whitewaist")

In [ ]:
cl = defaultdict(dict)
for fname in tqdm(list(map(id2whitewaist,  fl["out"]))):
    _id = fname2id(fname)
    arr = load_arr(fname)
    upper = arr[:256, :, :]
    lower = arr[256:, :, :]
    lm =     np.where(np.all(np.all(lower == 255, axis=2), axis=1))[0].min(initial=256)
    lu = 256-np.where(np.all(np.all(upper == 255, axis=2), axis=1))[0].max(initial=0)
    sp = (arr == 255).sum()
    cl["lm"][_id] = lm
    cl["lu"][_id] = lu
    cl["sp"][_id] = sp

In [ ]:
name = "v4_sm"
if name in Manager.MANAGED:
    del Manager.MANAGED[name]
Manager(map(id2whitewaist,  fl["out"]-Manager.allbag()), name=name, N=12*6-1, key = lambda _id: -cl["sp"][_id])

In [ ]:
name = "v4_lm"
if name in Manager.MANAGED:
    del Manager.MANAGED[name]
Manager(map(id2whitewaist,  fl["out"]-Manager.allbag()), name=name, N=12*6-1, key = cl["lm"].get)

In [ ]:
name = "v4_lm"
Manager.MANAGED[name].bag |= set(Manager.MANAGED[name].fnames[:(12*6-1)*50])

In [ ]:
name = "v4_lu"
if name in Manager.MANAGED:
    del Manager.MANAGED[name]
Manager(map(id2whitewaist,  fl["out"]-Manager.allbag()), name=name, N=12*6-1, key = cl["lu"].get)

In [ ]:
name = "v4_sp"
if name in Manager.MANAGED:
    del Manager.MANAGED[name]
Manager(map(id2whitewaist,  fl["out"]-Manager.allbag()), name=name, N=12*6-1, key = cl["sp"].get)

In [ ]:
name = "v4_sp"
Manager.MANAGED[name].bag |= set(Manager.MANAGED[name].fnames[:(12*6-1)*40])

In [ ]:
name = "v4_score"
if name in Manager.MANAGED:
    del Manager.MANAGED[name]
Manager(map(id2whitewaist,  fl["out"]-Manager.allbag()), name=name, N=12*6-1, key = id2score)

In [ ]:
Manager(map(id2whitewaist,  fl["out"]), name="v4_all", N=12*6-1)

In [ ]:
def f(_id):
    x = load_pkl(id2pkl(_id))
    return p["scores"][0]
Manager(map(id2whitewaist,  fl["out"]), name="v4_instance", N=12*6-1, key = f)

In [ ]:
def f(_id):
    x = load_json(id2json(_id))
    i = np.where(x.label == 1)[0]
    return x.score[i]
Manager(map(id2whitewaist,  fl["out"]), name="v4_face", N=12*6-1, key = f)

In [ ]:
fl["checked"] = fl["out"]-Manager.allbag()

In [ ]:
score = scoring(fl["checked"])
fl["score"] = score[1]

In [ ]:
flscore = list(fl["score"])
Random(0).shuffle(flscore)
options_train = list()
options_val = list()
for i, _id in enumerate(flscore):
    fname = id2fname(_id, prefix="whitewaist")
    if i < 80000:
        dname = id2fname(_id, prefix="v4")
        options_train.append(f"{fname}\t{dname}")
    else:
        dname = id2fname(_id, prefix="v4_val")
        options_val.append(f"{fname}\t{dname}")
split_dump(options_train, "option_v4")
split_dump(options_train, "option_v4_val")
prepare("v4")
prepare("v4_val")